Data Scraping Notebook

In [72]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd

# Import the API key
from config import geoapify_key
from config import census_key


# Launch the browser
browser = Browser('chrome', headless=False)

Create a "dictionary" list listing all states - Createing the list will help to make sure all states are listed and no state data will be lost in dataframe merges that will follow. 

In [73]:
# Set the URL to visit to a variable
url_states = "https://developers.google.com/public-data/docs/canonical/states_csv"

# Send the browser to the URL
browser.visit(url_states)

# Save the HTML from the browser
html = browser.html

# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, 'html.parser')

In [74]:
#find all divs that contain state marker
table_state = soup.find('div', class_='devsite-table-wrapper')

In [75]:
#Extract state-specific tax data from the table and build a dictionary of the data. 
state = {}

# Find the table element
state_list = []


# Check if the table is found
if table_state:
    # Find all rows in the table body
    rows = table_state.select('tbody tr')

    # Loop through the rows
    for row in rows:
        # Find all cells in the row
        cells = row.select('td')
        
        # Check if there are cells in the row
        if cells:
            # Extract data from the cells
            state = cells[0].text.strip()
            lat = cells[1].text.strip()
            lon = cells[2].text.strip()
            name = cells[3].text.strip()
            
            # Create a dictionary for the current state
            state = {'state_abbr': state, 'state': name}
            
            # Append the state data dictionary to the list
            state_list.append(state)

# Print the extracted data
#for state in state_list:
    #print(state)

In [76]:
#load scraped state data into dataframe
state_df = pd.DataFrame(state_list)
#save dataframe to csv
state_df.to_csv('../Resources/state_list_REFERENCE.csv', index=False)

1. Which states require payment of state taxes on gambling winnings?

 A table showing the latest information regarding which states tax gambling winnings. Keep in mind that each state that requires state tax on gambling income has its own tax rate and guidelines. In some cases, gambling winnings are subject to state tax except for lottery winnings.

In [77]:
# Set the URL to visit to a variable
url = "https://www.playusa.com/taxes/"

# Send the browser to the URL
browser.visit(url)

# Save the HTML from the browser
html = browser.html

# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, 'html.parser')

In [78]:
#find all divs that contain state marker
table = soup.find('table', class_='tablepress-responsive')

In [79]:
#Extract state-specific tax data from the table and build a dictionary of the data. 
state_tax = {}

# Find the table element
state_tax_list = []

# Find the table element
#table = soup.find('table', {'class': 'tablepress'})

# Check if the table is found
if table:
    # Find all rows in the table body
    rows = table.select('tbody tr')

    # Loop through the rows
    for row in rows:
        # Find all cells in the row
        cells = row.select('td')
        
        # Check if there are cells in the row
        if cells:
            # Extract data from the cells
            state = cells[0].text.strip()
            tax_status = cells[1].text.strip()
            
            # Create a dictionary for the current state
            state_data = {'state': state, 'tax_status': tax_status}
            
            # Append the state data dictionary to the list
            state_tax_list.append(state_data)

# Print the extracted data
for state_data in state_tax_list:
    print(state_data)

{'state': 'Alabama', 'tax_status': 'Yes'}
{'state': 'Alaska', 'tax_status': 'No'}
{'state': 'Arizona', 'tax_status': 'Yes'}
{'state': 'Arkansas', 'tax_status': 'Yes'}
{'state': 'California', 'tax_status': 'No'}
{'state': 'Colorado', 'tax_status': 'Yes'}
{'state': 'Connecticut', 'tax_status': 'Yes'}
{'state': 'Delaware', 'tax_status': 'Yes'}
{'state': 'D.C.', 'tax_status': 'Yes'}
{'state': 'Florida', 'tax_status': 'No'}
{'state': 'Georgia', 'tax_status': 'Yes'}
{'state': 'Hawaii', 'tax_status': 'No'}
{'state': 'Idaho', 'tax_status': 'Yes'}
{'state': 'Illinois', 'tax_status': 'Yes'}
{'state': 'Indiana', 'tax_status': 'Yes'}
{'state': 'Iowa', 'tax_status': 'Yes'}
{'state': 'Kansas', 'tax_status': 'Yes'}
{'state': 'Kentucky', 'tax_status': 'Yes'}
{'state': 'Louisiana', 'tax_status': 'Yes'}
{'state': 'Maine', 'tax_status': 'Yes'}
{'state': 'Maryland', 'tax_status': 'Yes'}
{'state': 'Massachusetts', 'tax_status': 'Yes'}
{'state': 'Michigan', 'tax_status': 'Yes'}
{'state': 'Minnesota', 'tax_s

In [80]:
#create a dataframe from the state_data dictionary of lists and display the first 5 rows
state_tax_df = pd.DataFrame(state_tax_list)
state_tax_df.head()

,state,tax_status
0,Alabama,Yes
1,Alaska,No
2,Arizona,Yes
3,Arkansas,Yes
4,California,No


In [81]:
#replace D.C. to District of Columbia in the state column
state_tax_df['state'] = state_tax_df['state'].replace('D.C.', 'District of Columbia')

Combining the tax information with the full list of states to ensure all states are listed for merges that follow. 

In [82]:
#merge state_df and state_tax_df
state_tax_df = state_df.merge(state_tax_df, on='state', how='left')

In [83]:
state_tax_df.to_csv('../Resources/state_tax_df_INTERMEDIATE_FILE.csv', index=False)

2. Scrape the running total for sports betting handle and revenue reported to date across all active markets:

In [84]:
# Set the URL to visit to a variable
url = "https://www.legalsportsreport.com/sports-betting/revenue/"

# Send the browser to the URL
browser.visit(url)

# Save the HTML from the browser
html = browser.html

# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, 'html.parser')

In [85]:
rev_table = soup.find('table', class_='tablepress-id-Rev_Markets')

In [86]:
# Create an empty list to store market data
market_data_list = []

# Find the table element
#rev_table = soup.find('table', {'class': 'tablepress'})

# Check if the table is found
if rev_table:
    # Find all rows in the table body
    rows = rev_table.select('tbody tr')

    # Loop through the rows
    for row in rows:
        # Find all cells in the row
        cells = row.select('td')
        
        # Check if there are cells in the row
        if cells:
            # Extract data from the cells
            market = cells[0].text.strip()
            handle = cells[1].text.strip()
            revenue = cells[2].text.strip()
            hold = cells[3].text.strip()
            taxes = cells[4].text.strip()
            
            # Create a dictionary for the current market
            market_data = {'state': market, 'handle': handle, 'revenue': revenue, 'hold': hold, 'taxes': taxes}
            
            # Append the market data dictionary to the list
            market_data_list.append(market_data)

# Print the extracted data
for market_data in market_data_list:
    print(market_data)

{'state': 'New Jersey', 'handle': '$39,461,272,422', 'revenue': '$2,871,419,711', 'hold': '7.3%', 'taxes': '$397,552,569'}
{'state': 'Nevada', 'handle': '$33,520,083,578', 'revenue': '$1,885,621,000', 'hold': '5.6%', 'taxes': '$127,279,418'}
{'state': 'New York', 'handle': '$25,545,018,522', 'revenue': '$2,231,885,589', 'hold': '8.7%', 'taxes': '$1,116,577,865'}
{'state': 'Illinois', 'handle': '$24,059,865,704', 'revenue': '$1,962,146,676', 'hold': '8.2%', 'taxes': '$312,219,908'}
{'state': 'Pennsylvania', 'handle': '$22,766,881,865', 'revenue': '$1,856,291,582', 'hold': '8.2%', 'taxes': '$462,263,411'}
{'state': 'Colorado', 'handle': '$12,795,525,493', 'revenue': '$857,637,396', 'hold': '6.7%', 'taxes': '$46,491,000'}
{'state': 'Indiana', 'handle': '$12,751,685,026', 'revenue': '$1,085,683,295', 'hold': '8.5%', 'taxes': '$103,236,082'}
{'state': 'Michigan', 'handle': '$11,268,699,327', 'revenue': '$972,698,437', 'hold': '8.6%', 'taxes': '$49,613,201'}
{'state': 'Arizona', 'handle': '$

In [87]:
market_rev_df = pd.DataFrame(market_data_list)
market_rev_df.head(5)

,state,handle,revenue,hold,taxes
0,New Jersey,"$39,461,272,422","$2,871,419,711",7.3%,"$397,552,569"
1,Nevada,"$33,520,083,578","$1,885,621,000",5.6%,"$127,279,418"
2,New York,"$25,545,018,522","$2,231,885,589",8.7%,"$1,116,577,865"
3,Illinois,"$24,059,865,704","$1,962,146,676",8.2%,"$312,219,908"
4,Pennsylvania,"$22,766,881,865","$1,856,291,582",8.2%,"$462,263,411"


In [88]:
#replace "?"" in market_rev_df for handle, revenue, hold, and taxes with 0
market_rev_df = market_rev_df.replace({'?': 0})

Combining the to date revenue information with the full list of states to ensure all states are listed for merges that follow. 

In [89]:
#merge state_df and market_rev_df by state
state_rev_df = state_df.merge(market_rev_df, on='state', how='left')
#state_rev_df.head()
state_rev_df.head(5)

,state_abbr,state,handle,revenue,hold,taxes
0,AK,Alaska,NaN,NaN,NaN,NaN
1,AL,Alabama,NaN,NaN,NaN,NaN
2,AR,Arkansas,"$486,217,898","$49,119,871",10.1%,"$7,134,908"
3,AZ,Arizona,"$11,005,985,362","$907,366,992",8.2%,"$51,509,590"
4,CA,California,NaN,NaN,NaN,NaN


In [90]:
#replace NaN with 0
state_rev_df = state_rev_df.fillna(0)

Adding the sports league detail to the revenue by state df.

In [91]:
#add sport league info to the market revenue data frame - data added from a csv file copied from WIKI https://en.wikipedia.org/wiki/Major_professional_sports_teams_of_the_United_States_and_Canada#:~:text=Teams%20%20%20%20Team%20%20%20,%20%20MLB%20%2018%20more%20rows%20
#file previously copied directly and saved in the csv format
#read the csv file
us_major_teams_df = pd.read_csv('../Resources/us_major_teams.csv')
us_major_teams_df.head()


,Team,Venue,City,State/Province,League
0,Arizona Diamondbacks,Chase Field,Phoenix,Arizona,MLB
1,Phoenix Suns,Footprint Center,Phoenix,Arizona,NBA
2,Arizona Cardinals,State Farm Stadium,Glendale,Arizona,NFL
3,Arizona Coyotes,Mullett Arena,Tempe,Arizona,NHL
4,Anaheim Ducks,Honda Center,Anaheim,California,NHL


In [92]:
# Create a summary DataFrame to count unique teams and leagues per state
summary_data = []

# Get unique states
unique_states = us_major_teams_df['State/Province'].unique()

for state in unique_states:
    state_teams = us_major_teams_df[us_major_teams_df['State/Province'] == state]
    total_teams = state_teams['Team'].nunique()
    
    state_summary = {'state': state, 'Teams Count': total_teams}
    
    # Count teams for each league
    leagues = ['MLB', 'NBA', 'NFL', 'NHL', 'MLS', 'NCAA']
    for league in leagues:
        league_count = state_teams[state_teams['League'] == league]['Team'].nunique()
        state_summary[league] = league_count
    
    summary_data.append(state_summary)

# Create the summary DataFrame
summary_df = pd.DataFrame(summary_data)

# Display the summary DataFrame
print(summary_df)



                   state  Teams Count  MLB  NBA  NFL  NHL  MLS  NCAA
0                Arizona            4    1    1    1    1    0     0
1             California           19    5    4    3    3    4     0
2               Colorado            5    1    1    1    1    1     0
3   District of Columbia            4    1    1    0    1    1     0
4                Florida           11    2    2    3    2    2     0
5                Georgia            4    1    1    1    0    1     0
6               Illinois            6    2    1    1    1    1     0
7                Indiana            2    0    1    1    0    0     0
8                 Kansas            1    0    0    0    0    1     0
9              Louisiana            2    0    1    1    0    0     0
10              Maryland            3    1    0    2    0    0     0
11         Massachusetts            5    1    1    1    1    1     0
12              Michigan            4    1    1    1    1    0     0
13             Minnesota          

In [93]:
#merge market_rev_df with summary_df with by state with join on state 
merged_state_rev_league_df = pd.merge(state_rev_df, summary_df, on='state', how='left')

# Display the merged DataFrame
merged_state_rev_league_df.head(5)


,state_abbr,state,handle,revenue,hold,taxes,Teams Count,MLB,NBA,NFL,NHL,MLS,NCAA
0,AK,Alaska,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AL,Alabama,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AR,Arkansas,"$486,217,898","$49,119,871",10.1%,"$7,134,908",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AZ,Arizona,"$11,005,985,362","$907,366,992",8.2%,"$51,509,590",4.00,1.00,1.00,1.00,1.00,0.00,0.00
4,CA,California,0,0,0,0,19.00,5.00,4.00,3.00,3.00,4.00,0.00


In [94]:
#show datatype for each column 
merged_state_rev_league_df.dtypes

state_abbr      object
state           object
handle          object
revenue         object
hold            object
taxes           object
Teams Count    float64
MLB            float64
NBA            float64
NFL            float64
NHL            float64
MLS            float64
NCAA           float64
dtype: object

In [95]:
# Remove '$' and ',' and convert to float
merged_state_rev_league_df['handle'] = merged_state_rev_league_df['handle'].str.replace('[\$,]', '', regex=True).astype(float)
merged_state_rev_league_df['revenue'] = merged_state_rev_league_df['revenue'].str.replace('[\$,]', '', regex=True).astype(float)
merged_state_rev_league_df['taxes'] = merged_state_rev_league_df['taxes'].str.replace('[\$,]', '', regex=True).astype(float)

# Set the float format
pd.options.display.float_format = '{:.2f}'.format

In [96]:
def convert_percentage(percentage_value):
    if isinstance(percentage_value, str) and percentage_value != '':
        return float(percentage_value.replace('%', '')) / 100
    else:
        return 0.0  # Handle empty strings and non-string values by returning 0.0

merged_state_rev_league_df['hold'] = merged_state_rev_league_df['hold'].apply(convert_percentage)

In [97]:
#Datframe cleanup before loading into csv:
merged_state_rev_league_df = merged_state_rev_league_df.fillna(0)

In [98]:
# Display the merged DataFrame
merged_state_rev_league_df.head(5)

,state_abbr,state,handle,revenue,hold,taxes,Teams Count,MLB,NBA,NFL,NHL,MLS,NCAA
0,AK,Alaska,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,AL,Alabama,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,AR,Arkansas,486217898.00,49119871.00,0.10,7134908.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,AZ,Arizona,11005985362.00,907366992.00,0.08,51509590.00,4.00,1.00,1.00,1.00,1.00,0.00,0.00
4,CA,California,0.00,0.00,0.00,0.00,19.00,5.00,4.00,3.00,3.00,4.00,0.00


In [99]:
merged_state_rev_league_df.to_csv('../Resources/market_rev_teams_df_FINALSET.csv', index=False)

3. US sports betting revenue by month

In [100]:
month_rev_table = soup.find('table', class_='tablepress-id-Rev_Months')

In [101]:
# Create an empty list to store market data
month_data_list = []

# Find the table element
#rev_table = soup.find('table', {'class': 'tablepress'})

# Check if the table is found
if month_rev_table:
    # Find all rows in the table body
    rows = month_rev_table.select('tbody tr')

    # Loop through the rows
    for row in rows:
        # Find all cells in the row
        cells = row.select('td')
        
        # Check if there are cells in the row
        if cells:
            # Extract data from the cells
            month = cells[0].text.strip()
            handle = cells[1].text.strip()
            revenue = cells[2].text.strip()
            hold = cells[3].text.strip()
            taxes = cells[4].text.strip()
            
            # Create a dictionary for the current market
            timeline_data = {'month': month, 'handle': handle, 'revenue': revenue, 'hold': hold, 'taxes': taxes}
            
            # Append the market data dictionary to the list
            month_data_list.append(timeline_data)

# Print the extracted data
for timeline_data in month_data_list:
    print(timeline_data)

{'month': 'June 2018', 'handle': '$309,695,659', 'revenue': '$24,630,915', 'hold': '8.0%', 'taxes': '$2,221,650'}
{'month': 'July 2018', 'handle': '$294,006,820', 'revenue': '$8,433,994', 'hold': '2.9%', 'taxes': '$926,804'}
{'month': 'August 2018', 'handle': '$359,275,692', 'revenue': '$23,625,881', 'hold': '6.6%', 'taxes': '$2,524,419'}
{'month': 'September 2018', 'handle': '$817,681,310', 'revenue': '$95,358,434', 'hold': '11.7%', 'taxes': '$12,245,073'}
{'month': 'October 2018', 'handle': '$854,014,077', 'revenue': '$45,832,391', 'hold': '5.4%', 'taxes': '$5,217,752'}
{'month': 'November 2018', 'handle': '$996,071,884', 'revenue': '$54,349,323', 'hold': '5.5%', 'taxes': '$6,272,898'}
{'month': 'December 2018', 'handle': '$990,933,059', 'revenue': '$82,266,923', 'hold': '8.3%', 'taxes': '$11,262,831'}
{'month': 'January 2019', 'handle': '$1,002,092,655', 'revenue': '$43,052,327', 'hold': '4.3%', 'taxes': '$6,529,559'}
{'month': 'February 2019', 'handle': '$881,756,490', 'revenue': '

In [102]:
month_rev_data_df = pd.DataFrame(month_data_list)
month_rev_data_df.head(5)

,month,handle,revenue,hold,taxes
0,June 2018,"$309,695,659","$24,630,915",8.0%,"$2,221,650"
1,July 2018,"$294,006,820","$8,433,994",2.9%,"$926,804"
2,August 2018,"$359,275,692","$23,625,881",6.6%,"$2,524,419"
3,September 2018,"$817,681,310","$95,358,434",11.7%,"$12,245,073"
4,October 2018,"$854,014,077","$45,832,391",5.4%,"$5,217,752"


In [103]:
#data cleaning before saving the csv file
# Remove '$' and ',' and convert to float
month_rev_data_df['handle'] = month_rev_data_df['handle'].str.replace('[\$,]', '', regex=True).astype(float)
month_rev_data_df['revenue'] = month_rev_data_df['revenue'].str.replace('[\$,]', '', regex=True).astype(float)
month_rev_data_df['taxes'] = month_rev_data_df['taxes'].str.replace('[\$,]', '', regex=True).astype(float)

# Set the float format
pd.options.display.float_format = '{:.2f}'.format

In [104]:
def convert_percentage(percentage_value):
    if isinstance(percentage_value, str) and percentage_value != '':
        return float(percentage_value.replace('%', '')) / 100
    else:
        return 0.0  # Handle empty strings and non-string values by returning 0.0

month_rev_data_df['hold'] = month_rev_data_df['hold'].apply(convert_percentage)

In [105]:
month_rev_data_df.head(5)

,month,handle,revenue,hold,taxes
0,June 2018,309695659.00,24630915.00,0.08,2221650.00
1,July 2018,294006820.00,8433994.00,0.03,926804.00
2,August 2018,359275692.00,23625881.00,0.07,2524419.00
3,September 2018,817681310.00,95358434.00,0.12,12245073.00
4,October 2018,854014077.00,45832391.00,0.05,5217752.00


In [106]:
month_rev_data_df.to_csv('../Resources/month_rev_each_state_df_FINALSET.csv', index=False)

6. Summary by online vs in-person betting by state - https://www.gamingtoday.com/states/ 

In [107]:
# Set the URL to visit to a variable
url_legal = "https://www.gamingtoday.com/states/"

# Send the browser to the URL
browser.visit(url_legal)

# Save the HTML from the browser
html = browser.html

# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, 'html.parser')

In [108]:
#find all divs that contain state marker
table_legal = soup.find('table', class_='tablepress-id-1781')


In [116]:
# Create an empty list to store market data
legal_data_list = []

# Find the table element
#rev_table = soup.find('table', {'class': 'tablepress'})

# Check if the table is found
if table_legal:
    # Find all rows in the table body
    rows = table_legal.select('tbody tr')

    # Loop through the rows
    for row in rows:
        # Find all cells in the row
        cells = row.select('td')
        
        # Check if there are cells in the row
        if cells:
            # Extract data from the cells
            state = cells[0].text.strip()
            online = cells[1].text.strip()
            in_person = cells[2].text.strip()
            
            # Create a dictionary for the current market
            legal_data = {'state': state, 'online': online, 'in_person': in_person}
            
            # Append the market data dictionary to the list
            legal_data_list.append(legal_data)

# Print the extracted data
for legal_data in legal_data_list:
    print(legal_data)

{'state': 'Alabama Sports Betting', 'online': 'X', 'in_person': 'X'}
{'state': 'Alaska Sports Betting', 'online': 'X', 'in_person': 'X'}
{'state': 'Arizona Sports Betting', 'online': '✓', 'in_person': '✓'}
{'state': 'Arkansas Sports Betting', 'online': '✓', 'in_person': '✓'}
{'state': 'California Sports Betting', 'online': 'X', 'in_person': 'X'}
{'state': 'Colorado Sports Betting', 'online': '✓', 'in_person': '✓'}
{'state': 'Connecticut Sports Betting', 'online': '✓', 'in_person': '✓'}
{'state': 'Delaware Sports Betting', 'online': 'X', 'in_person': '✓'}
{'state': 'Florida Sports Betting', 'online': 'X', 'in_person': 'X'}
{'state': 'Georgia Sports Betting', 'online': 'X', 'in_person': 'X'}
{'state': 'Hawaii Sports Betting', 'online': 'X', 'in_person': 'X'}
{'state': 'Idaho Sports Betting', 'online': 'X', 'in_person': 'X'}
{'state': 'Illinois Sports Betting', 'online': '✓', 'in_person': '✓'}
{'state': 'Indiana Sports Betting', 'online': '✓', 'in_person': '✓'}
{'state': 'Iowa Sports Bett

In [117]:
legal_data_list_df = pd.DataFrame(legal_data_list)
legal_data_list_df.head(5)

,state,online,in_person
0,Alabama Sports Betting,X,X
1,Alaska Sports Betting,X,X
2,Arizona Sports Betting,✓,✓
3,Arkansas Sports Betting,✓,✓
4,California Sports Betting,X,X


In [118]:
#replace 'X' to 'NO' and '✓' with 'YES' in all columns of the legal_data_list_df 
legal_data_list_df = legal_data_list_df.replace('X', 'NO')
legal_data_list_df = legal_data_list_df.replace('✓', 'YES')
legal_data_list_df.head(5)


,state,online,in_person
0,Alabama Sports Betting,NO,NO
1,Alaska Sports Betting,NO,NO
2,Arizona Sports Betting,YES,YES
3,Arkansas Sports Betting,YES,YES
4,California Sports Betting,NO,NO


In [119]:
#remove "Sports Betting" from the state column
legal_data_list_df['state'] = legal_data_list_df['state'].str.replace('Sports Betting', '')
legal_data_list_df.head(5)

,state,online,in_person
0,Alabama,NO,NO
1,Alaska,NO,NO
2,Arizona,YES,YES
3,Arkansas,YES,YES
4,California,NO,NO


In [120]:
#create additional column "Ways to bet" and po pulatie with "Online-Only" if online is YES and in_person is NO
legal_data_list_df['Ways_to_bet'] = legal_data_list_df.apply(lambda x: 
    'Not Legal' if x['online'] == 'NO' and x['in_person'] == 'NO' else
    'Online Only' if x['online'] == 'YES' and x['in_person'] == 'NO' else
    'In Person Only' if x['online'] == 'NO' and x['in_person'] == 'YES' else
    'Online & In-Person' if x['online'] == 'YES' and x['in_person'] == 'YES' else
    'Pending' if 'Pending' in (x['online'], x['in_person']) else
    '',
    axis=1)

legal_data_list_df.head(5)

,state,online,in_person,Ways_to_bet
0,Alabama,NO,NO,Not Legal
1,Alaska,NO,NO,Not Legal
2,Arizona,YES,YES,Online & In-Person
3,Arkansas,YES,YES,Online & In-Person
4,California,NO,NO,Not Legal


In [121]:
#remove spaces after state name in the state column
legal_data_list_df['state'] = legal_data_list_df['state'].str.strip()

In [123]:
#replace 'Washington D.C.' to 'District of Columbia' in the state column
legal_data_list_df['state'] = legal_data_list_df['state'].replace('Washington, D.C.', 'District of Columbia')
#replace 'Washington State' to 'Washington' in the state column
legal_data_list_df['state'] = legal_data_list_df['state'].replace('Washington State', 'Washington')


In [124]:
legal_data_list_df.to_csv('../Resources/legal_online_inperson_FINALSET.csv', index=False)

In [115]:
browser.quit()